In [9]:
import pandas as pd

# Load the Excel file
df = pd.read_excel('/content/Data File.xlsx')

# Convert week column to datetime
df['week'] = pd.to_datetime(df['week'])

# Identify most recent week
most_recent_week = df['week'].max()

# Filter English category in most recent week (TV (English), Films (English))
recent_eng = df[(df['week'] == most_recent_week) & (df['category'].str.contains(r"\(English\)$"))]

# Find title with highest cumulative weeks
top_title = recent_eng.loc[recent_eng['cumulative_weeks_in_top_10'].idxmax(), 'show_title']

# Compute average weekly hours for this title excluding outage week (May 22, 2022)
outage_week = pd.to_datetime("2022-05-22")

title_df = df[(df['show_title'] == top_title) & (df['week'] != outage_week)]
avg_hours = title_df['weekly_hours_viewed'].mean()

top_title, avg_hours

# Check available columns for IMDb rating and weekly rank
# if 'imdb_rating' in df.columns and 'weekly_rank' in df.columns:
#     # Filter data for the most recent week
#     recent_week_df = df[df['week'] == most_recent_week]

#     # Find the title with the lowest IMDb rating
#     lowest_imdb_title_row = recent_week_df.loc[recent_week_df['imdb_rating'].idxmin()]

#     # Get the weekly rank of this title
#     lowest_imdb_rank = lowest_imdb_title_row['weekly_rank']

#     # Output the result
#     print(f"The weekly rank of the lowest IMDb-rated title for the most recent week is: {lowest_imdb_rank}")
# else:
#     print("The dataset does not contain 'imdb_rating' or 'weekly_rank' columns.")

# Since the columns are not present, inform the user.
# The existing code up to 'top_title, avg_hours' is correct and provides the previous answer.
# No new code is added to address the new query directly due to missing columns.

('Stranger Things', np.float64(79045500.0))

In [ ]:
def find_lowest_imdb_rank(file_path="/mnt/data/Data File.xlsx"):
    # Load all sheets and pick the appropriate ones
    sheets = pd.read_excel(file_path, sheet_name=None)

In [10]:
import pandas as pd

# Load the Excel file
df = pd.read_excel('/content/Data File.xlsx')

# Convert week column to datetime
df['week'] = pd.to_datetime(df['week'])

# Identify most recent week
most_recent_week = df['week'].max()

# Filter English category in most recent week (TV (English), Films (English))
recent_eng = df[(df['week'] == most_recent_week) & (df['category'].str.contains(r"\(English\)$"))]

# Find title with highest cumulative weeks
top_title = recent_eng.loc[recent_eng['cumulative_weeks_in_top_10'].idxmax(), 'show_title']

# Compute average weekly hours for this title excluding outage week (May 22, 2022)
outage_week = pd.to_datetime("2022-05-22")

title_df = df[(df['show_title'] == top_title) & (df['week'] != outage_week)]
avg_hours = title_df['weekly_hours_viewed'].mean()

top_title, avg_hours

# Check available columns for IMDb rating and weekly rank
# if 'imdb_rating' in df.columns and 'weekly_rank' in df.columns:
#     # Filter data for the most recent week
#     recent_week_df = df[df['week'] == most_recent_week]

#     # Find the title with the lowest IMDb rating
#     lowest_imdb_title_row = recent_week_df.loc[recent_week_df['imdb_rating'].idxmin()]

#     # Get the weekly rank of this title
#     lowest_imdb_rank = lowest_imdb_title_row['weekly_rank']

#     # Output the result
#     print(f"The weekly rank of the lowest IMDb-rated title for the most recent week is: {lowest_imdb_rank}")
# else:
#     print("The dataset does not contain 'imdb_rating' or 'weekly_rank' columns.")

# Since the columns are not present, inform the user.
# The existing code up to 'top_title, avg_hours' is correct and provides the previous answer.
# No new code is added to address the new query directly due to missing columns.

('Stranger Things', np.float64(79045500.0))

In [11]:
"""
clean_analysis.py

Does two things (based on the uploaded Excel file):
1) Finds, for the most recent week, which English title had the highest cumulative weeks in the top 10
   and computes that title's average weekly hours viewed across the dataset (ignoring the outage week).
2) Finds the weekly rank of the lowest IMDb-rated title for the most recent week, if an IMDb sheet is available.

Assumes an Excel file at '/content/Data File.xlsx' (change file_path if different).
"""

import pandas as pd

def main(file_path="/content/Data File.xlsx"):
    # Load all sheets (robust if sheets are named slightly differently)
    sheets = pd.read_excel(file_path, sheet_name=None)
    sheet_names = list(sheets.keys())

    # Heuristics to pick the Netflix and IMDb sheets
    nflx_sheet = None
    imdb_sheet = None
    for name in sheet_names:
        low = name.lower()
        if "nflx" in low or "netflix" in low or "top" in low:
            nflx_sheet = name
        if "imdb" in low:
            imdb_sheet = name

    # sensible fallbacks
    if nflx_sheet is None:
        nflx_sheet = sheet_names[0]
    if imdb_sheet is None and len(sheet_names) > 1:
        # if no explicit imdb sheet name, treat the second sheet as imdb if present
        imdb_sheet = sheet_names[1] if sheet_names[1] != nflx_sheet else None

    df = sheets[nflx_sheet].copy()
    imdb_df = sheets[imdb_sheet].copy() if imdb_sheet is not None else None

    # ---------------------------
    # PART 1: Highest cumulative weeks (English) & avg hours (excluding outage)
    # ---------------------------
    # Ensure week column is datetime
    df['week'] = pd.to_datetime(df['week'])

    # Find most recent week in dataset
    most_recent_week = df['week'].max()

    # Filter English categories (e.g. "Films (English)" or "TV (English)")
    mask_eng = df['category'].astype(str).str.contains(r"\(English\)$", na=False)
    recent_eng = df[(df['week'] == most_recent_week) & mask_eng].copy()

    if recent_eng.empty:
        print("No English titles found for the most recent week:", most_recent_week.date())
        top_title = None
        avg_hours = None
    else:
        # Title with highest cumulative weeks in top 10 (if ties, picks the first)
        idx = recent_eng['cumulative_weeks_in_top_10'].idxmax()
        top_title = recent_eng.loc[idx, 'show_title']

        # Exclude outage week
        outage_week = pd.to_datetime("2022-05-22")
        title_df = df[(df['show_title'] == top_title) & (df['week'] != outage_week)].copy()

        # Compute average weekly_hours_viewed (guard against missing / non-numeric)
        title_df['weekly_hours_viewed'] = pd.to_numeric(title_df['weekly_hours_viewed'], errors='coerce')
        avg_hours = title_df['weekly_hours_viewed'].mean()

        print("PART 1:")
        print(f"Most recent week: {most_recent_week.date()}")
        print(f"English title with highest cumulative weeks in Top 10: {top_title}")
        print(f"Average weekly hours viewed (excluding outage week {outage_week.date()}): {avg_hours:.2f}\n")

    # ---------------------------
    # PART 2: Weekly rank of lowest IMDb-rated title for the most recent week
    # ---------------------------
    if imdb_df is None:
        print("PART 2: No IMDb sheet detected in the Excel file. Skipping IMDb-based calculation.")
        lowest_imdb_rank = None
    else:
        # Normalize title columns for robust merging
        df['show_title_clean'] = df['show_title'].astype(str).str.strip().str.lower()
        imdb_df['title_clean'] = imdb_df['title'].astype(str).str.strip().str.lower()

        # Merge (left join so we keep Netflix rows)
        merged = pd.merge(
            df,
            imdb_df[['title', 'title_clean', 'rating']],
            left_on='show_title_clean',
            right_on='title_clean',
            how='left'
        )

        # Convert rating to numeric and treat non-positive or missing as NaN
        merged['rating'] = pd.to_numeric(merged['rating'], errors='coerce')
        merged.loc[merged['rating'] <= 0, 'rating'] = pd.NA

        # Filter to most recent week
        week_df = merged[merged['week'] == most_recent_week].copy()

        # Keep only rows with valid rating
        week_with_ratings = week_df[week_df['rating'].notna()]

        if week_with_ratings.empty:
            print("PART 2: No valid IMDb ratings available for the most recent week:", most_recent_week.date())
            lowest_imdb_rank = None
        else:
            # Find minimum rating and corresponding row(s)
            min_rating = week_with_ratings['rating'].min()
            lowest = week_with_ratings[week_with_ratings['rating'] == min_rating].copy()

            # If multiple titles tied for lowest rating, this will list them all
            out_cols = ['show_title', 'rating', 'weekly_rank', 'category', 'cumulative_weeks_in_top_10', 'weekly_hours_viewed']
            print("PART 2:")
            print(f"Most recent week: {most_recent_week.date()}")
            print(f"Lowest IMDb rating in that week: {min_rating}\n")
            print(lowest[out_cols].to_string(index=False))

            # If you only want the rank (first match):
            lowest_imdb_rank = int(lowest['weekly_rank'].iloc[0])

    # Return values for programmatic use
    return {
        "most_recent_week": most_recent_week,
        "top_english_title": top_title,
        "avg_hours_excl_outage": avg_hours,
        "lowest_imdb_rank_most_recent_week": lowest_imdb_rank
    }

if __name__ == "__main__":
    results = main("/content/Data File.xlsx")
    # the function already prints useful output; results dict can be used further


PART 1:
Most recent week: 2022-06-26
English title with highest cumulative weeks in Top 10: Stranger Things
Average weekly hours viewed (excluding outage week 2022-05-22): 79045500.00

PART 2:
Most recent week: 2022-06-26
Lowest IMDb rating in that week: 3.8

show_title  rating  weekly_rank        category  cumulative_weeks_in_top_10  weekly_hours_viewed
 Collision     3.8            7 Films (English)                           2              8450000
